In [9]:
import pandas as pd
import numpy as np
import sys
import heapq

In [247]:
class Node:
    # initialises the Node class with the following attributes
    def __init__(self, node_id):
        self.id = node_id
        self.adjacent = {}
        self.distance = sys.maxsize
        self.visited = False
        self.parent_node = None
    
    # adds an adjacent node with a given weight to the interaction
    def add_adjacent(self, adjacent_node, weight = 1):
        self.adjacent[adjacent_node] = weight
    
    def get_weight(self, adjacent_node):
        return self.adjacent[adjacent_node]
    
    # sets the distance from the source node to a given node
    def set_distance(self, dist):
        self.distance = dist

    # sets the parent node
    def set_parentnode(self, parent_node):
        self.parent_node = parent_node
    
    # indicates if the node has been visited by the algorithm
    def set_visited(self):
        self.visited = True
    
    # sets the way way to compare classes
    def __le__(self, other):
        return self.distance <= other.distance
    
    def __lt__(self, other):
        return self.distance < other.distance

class Network:
    def __init__(self, sourcenodes):
        self.source_nodes = sourcenodes
        self.target_nodes = []
        self.net = pd.DataFrame()
    
    def add_targetnodes(self, node_id):
        self.target_nodes.append(node_id)
    
class Graph:

    def __init__(self):
        self.nodes_dict = {}
        self.num_nodes = 0
        self.subnetwork = ()
    
    def __iter__(self):
        return iter(self.nodes_dict.values())

    # creates a Node object and appends it to the nodes dictionary in the Graph class
    # it also counts up the number of nodes in the graph
    def add_node(self, node_id):
        self.num_nodes =+ 1
        new_node = Node(node_id)
        self.nodes_dict[node_id] = new_node
        return new_node
    
    def get_node(self, node_id):
        if node_id in self.nodes_dict:
            return self.nodes_dict[node_id]
        else:
            print('The node {} could not be found in the network!'.format(node_id))
            return None
    
    # adds an edge between a source node and a target node, and a weight ruling that interaction
    # If a Node class is not available for that node name, it creates one.
    # last, it adds to the adjacent attribute from the source Node class a dictionary item containing the target node id and the weight.
    def add_edge(self, source, target, weight = 1):
        if source not in self.nodes_dict:
            self.add_node(source)
        if target not in self.nodes_dict:
            self.add_node(target)
        
        self.nodes_dict[source].add_adjacent(self.nodes_dict[target], weight)
    
    def read_sif(self, path):
        network_df = pd.read_csv(path, sep='\t')
        for index, row in network_df.iterrows():
            self.add_edge(row['source'], row['target'], row['weight'])
    
    def dijkstra_solver(self, start_node):
        # Set the distance for the start node to zero 
        start_node.distance = 0

        # Priority queue, it gets the node with the smallest distance from the list within the unvisited nodes
        unvisited_queue = [(node.distance, node) for node in self]
        heapq.heapify(unvisited_queue)

        while len(unvisited_queue):
            # gets the node with the smallest distance 
            clostest_node = heapq.heappop(unvisited_queue)
            current_node = clostest_node[1]
            current_node.set_visited()

            # checks the adjacent nodes, doesn't recheck if visited.
            # it relaxes the edges by checking if there are quicker paths to a node than the ones already explored
            for adjacent_node in current_node.adjacent:
                if adjacent_node.visited:
                    continue
                new_dist = current_node.distance + current_node.get_weight(adjacent_node)
                
                if new_dist < adjacent_node.distance:
                    adjacent_node.set_distance(new_dist)
                    adjacent_node.parent_node = current_node

            # Rebuild the quieue
            # empty the list
            while len(unvisited_queue):
                heapq.heappop(unvisited_queue)
            # resets the queue
            unvisited_queue = [(node.distance, node) for node in self if not node.visited]
            heapq.heapify(unvisited_queue)
    
    # it gets a node, gets the name of the parent and appends it to a tuple. 
    def shortest_path(self, node, path):
        if node.parent_node:
            path.append(node.parent_node.id)
            self.shortest_path(node.parent_node, path)
        return
    
    def get_subnetwork(self, start_ids, downstream_nodes):
        self.subnetwork = Network(start_ids)
        for start_id in start_ids:
            start_node = self.get_node(start_id)
            self.dijkstra_solver(start_node)
            # 
            for node in downstream_nodes:
                target = self.get_node(node)
                if target is None: continue
                if node not in self.subnetwork.target_nodes: self.subnetwork.add_targetnodes(node) 
                path = [target.id]
                self.shortest_path(target, path)
                path.reverse()
                for i in range(0, len(path)-1):
                    new_row = pd.Series({'source': path[i], 'int': 'N', 'target': path[i+1]})
                    self.subnetwork.net = pd.concat([self.subnetwork.net, new_row.to_frame().T])
            self.subnetwork.net.drop_duplicates(inplace=True)



## Showcasing

Here we used toy data

In [232]:
pd.read_csv('toy_net.sif', sep='\t')

,source,target,weight
0,a,b,7
1,a,c,9
2,a,f,14
3,b,c,10
4,b,d,15
5,c,d,11
6,c,f,2
7,d,e,6
8,e,f,9


In [233]:
g = Graph()
g.read_sif('toy_net.sif')
downstream_nodes = ['e', 'd', 'q']


In [234]:
g.get_subnetwork('a', downstream_nodes)

The node q could not be found in the network!


In [238]:
g.subnetwork.target_nodes

['e', 'd']

Now, applied to the real scenario from the Trynska dataset:

In [250]:
g2 = Graph()

In [251]:
g2.read_sif('network_collectri.sif')
upstream_nodes = ['TGFB1','TGFB3','TGFB2', 'IL2']
downstream_nodes = pd.read_csv('downstream_hits.tsv', sep='\t', header=None)[0].values.tolist()

In [252]:
g2.get_subnetwork(upstream_nodes, downstream_nodes)

The node SCCPDH could not be found in the network!
The node IDS could not be found in the network!
The node OCIAD2 could not be found in the network!
The node PLP2 could not be found in the network!
The node TM2D3 could not be found in the network!
The node SETX could not be found in the network!
The node TATDN1 could not be found in the network!
The node FAM195B could not be found in the network!
The node SYNRG could not be found in the network!
The node VPS13C could not be found in the network!
The node PPCS could not be found in the network!
The node PVT1 could not be found in the network!
The node BIN2 could not be found in the network!
The node FAM207A could not be found in the network!
The node MRPL40 could not be found in the network!
The node GIMAP8 could not be found in the network!
The node NUDT4 could not be found in the network!
The node FNBP1L could not be found in the network!
The node RALGAPA2 could not be found in the network!
The node SLTM could not be found in the net

In [254]:
len(g2.subnetwork.target_nodes)

49

In [255]:
g2.subnetwork.net

,source,int,target
0,TGFB1,N,VTN
0,VTN,N,SERPINE1
0,SERPINE1,N,ELANE
0,ELANE,N,F2RL1
0,F2RL1,N,GNAQ
...,...,...,...
0,SSRP1,N,PRKCQ
0,PRKCQ,N,RASGRP3
0,CASP3,N,BIRC6
0,ATF6,N,PPARGC1A


In [256]:
g2.subnetwork.source_nodes

['TGFB1', 'TGFB3', 'TGFB2', 'IL2']